In [14]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
import os

import glob as gb

from sklearn.model_selection import train_test_split


import cv2
import random

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau



In [3]:
all_letters = os.listdir('C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small')

print(f'We have {len(all_letters)} letters , which are : {all_letters}')

We have 10 letters , which are : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [6]:
total_images = 0
for letter in all_letters : 
    available_images = gb.glob(pathname= f'C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small/{letter}/*.png')
    total_images+=len(available_images)
    print(f'for letter {letter} we have  {len(available_images)} available images')
print('-----------------------')    
print(f'Total Images are {total_images} images')

for letter A we have  1873 available images
for letter B we have  1873 available images
for letter C we have  1873 available images
for letter D we have  1873 available images
for letter E we have  1873 available images
for letter F we have  1873 available images
for letter G we have  1872 available images
for letter H we have  1872 available images
for letter I we have  1872 available images
for letter J we have  1872 available images
-----------------------
Total Images are 18726 images


In [7]:
Datadir = 'C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small'
Categories = ['A', 'D', 'I', 'G', 'H', 'B', 'F', 'C', 'J', 'E']

img_size = 80
training_data = []

for Category in Categories:
    path = os.path.join(Datadir, Category)
    print("Loading data from ", path)
    class_num = Categories.index(Category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(
                path, img), cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img_array, (img_size, img_size))
            training_data.append([img_resize, class_num])
        except Exception as e:
            pass

Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\A
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\D
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\I
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\G
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\H
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\B
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\F
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\C
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\J
Loading data from  C:/Users/SergS/Study/DS04/DS04_2/HW41/notMNIST_small\E


In [8]:
random.shuffle(training_data)

X, y = [], []
for feature, label in training_data:
    X.append(feature)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 1)
X = X / 255.0

y = to_categorical(y, num_classes = 10)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [10]:
print("full data:  ",X.shape)
print("Train:      ",X_train.shape)
print("Validation: ",X_val.shape)
print("Test:       ",X_test.shape)

full data:   (18724, 80, 80, 1)
Train:       (15165, 80, 80, 1)
Validation:  (1686, 80, 80, 1)
Test:        (1873, 80, 80, 1)


In [11]:
X_part, X_cv, y_part, y_cv = train_test_split(X, y, test_size=0.15, random_state=44, shuffle =True)

print('X_train shape is ' , X_part.shape)
print('X_test shape is ' , X_cv.shape)
print('y_train shape is ' , y_part.shape)
print('y_test shape is ' , y_cv.shape)

X_train shape is  (15915, 80, 80, 1)
X_test shape is  (2809, 80, 80, 1)
y_train shape is  (15915, 10)
y_test shape is  (2809, 10)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_part, y_part, test_size=0.25, random_state=44, shuffle =True)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

X_train shape is  (11936, 80, 80, 1)
X_test shape is  (3979, 80, 80, 1)
y_train shape is  (11936, 10)
y_test shape is  (3979, 10)


In [13]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = 4,  activation = tf.nn.relu , padding = 'same'),
        keras.layers.MaxPool2D(pool_size=(3,3), strides=None, padding='valid'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=32, kernel_size=4,activation = tf.nn.relu , padding='same'),
        keras.layers.MaxPool2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(filters=64, kernel_size=5,activation = tf.nn.relu , padding='same'),
        keras.layers.MaxPool2D(),
        keras.layers.Flatten(),    
        keras.layers.Dropout(0.5),        
        keras.layers.Dense(64),    
        keras.layers.Dropout(0.3),            
        keras.layers.Dense(units= 10,activation = tf.nn.softmax ),                

    ])
    

KerasModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [15]:
es_callback = EarlyStopping(patience=5, verbose=1, restore_best_weights=True)

In [17]:
#Train
KerasModel.fit(X_train,y_train,validation_data=(X_cv, y_cv),epochs=30,batch_size=64,verbose=1, callbacks=[es_callback])

Epoch 1/30
187/187 [==============================] - 2s 9ms/step - loss: 0.1509 - acc: 0.9535 - val_loss: 0.1976 - val_acc: 0.9455
Epoch 2/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1290 - acc: 0.9583 - val_loss: 0.2217 - val_acc: 0.9438
Epoch 3/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1281 - acc: 0.9592 - val_loss: 0.1848 - val_acc: 0.9480
Epoch 4/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1145 - acc: 0.9645 - val_loss: 0.2105 - val_acc: 0.9430
Epoch 5/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1224 - acc: 0.9607 - val_loss: 0.2078 - val_acc: 0.9430
Epoch 6/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1133 - acc: 0.9645 - val_loss: 0.2776 - val_acc: 0.9366
Epoch 7/30
187/187 [==============================] - 1s 8ms/step - loss: 0.1039 - acc: 0.9682 - val_loss: 0.2281 - val_acc: 0.9470
Epoch 8/30
187/187 [==============================] - 1s 8ms/step - loss: 0.

In [41]:
KerasModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                        

In [18]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

125/125 [==============================] - 1s 4ms/step
Prediction Shape is (3979, 10)


In [19]:
Letters ={0:'A', 1:'B' , 2:'C' ,3:'D' ,4:'E' ,5:'F' ,6:'G' ,7:'H' ,8:'I' ,9:'J' }

for i in list(np.random.randint(0,len(X_test) ,size= 10)) : 
    print(f'for sample  {i}  the predicted value is   {Letters[np.argmax(y_pred[i])]}   , while the actual letter is {Letters[np.argmax(y_test[i])]}')

for sample  1502  the predicted value is   B   , while the actual letter is B
for sample  929  the predicted value is   A   , while the actual letter is A
for sample  1387  the predicted value is   B   , while the actual letter is B
for sample  3538  the predicted value is   F   , while the actual letter is F
for sample  3528  the predicted value is   B   , while the actual letter is B
for sample  784  the predicted value is   B   , while the actual letter is B
for sample  185  the predicted value is   E   , while the actual letter is E
for sample  3514  the predicted value is   H   , while the actual letter is H
for sample  1710  the predicted value is   G   , while the actual letter is G
for sample  960  the predicted value is   C   , while the actual letter is C


In [20]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

125/125 [==============================] - 1s 5ms/step - loss: 0.1721 - acc: 0.9512
Test Loss is 0.17210520803928375
Test Accuracy is 0.951244056224823
